In [1]:
from datetime import date
from pandas import json_normalize
import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
df_work_raw = catalog.load('raw/openalex/work#parquet')

                    INFO     Loading data from raw/openalex/work#parquet (ParquetDataset)...    ]8;id=327834;file:///home/pablo/dev/SEDICI/dw/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=762523;file:///home/pablo/dev/SEDICI/dw/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#482\482]8;;\

In [3]:
df_work_raw = df_work_raw.convert_dtypes()
df_work_raw = df_work_raw.loc[:,['id','locations']]

In [4]:
df_work_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150 entries, 0 to 24
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         150 non-null    string
 1   locations  150 non-null    object
dtypes: object(1), string(1)
memory usage: 3.5+ KB


In [5]:
df_work_raw

,id,locations
0,https://openalex.org/W4211046286,"[{'is_accepted': True, 'is_oa': True, 'is_publ..."
1,https://openalex.org/W4292403095,"[{'is_accepted': True, 'is_oa': True, 'is_publ..."
2,https://openalex.org/W2098406858,"[{'is_accepted': False, 'is_oa': False, 'is_pu..."
3,https://openalex.org/W2501864044,"[{'is_accepted': True, 'is_oa': True, 'is_publ..."
4,https://openalex.org/W2012331775,"[{'is_accepted': True, 'is_oa': True, 'is_publ..."
...,...,...
20,https://openalex.org/W2972599699,"[{'is_accepted': True, 'is_oa': True, 'is_publ..."
21,https://openalex.org/W3024722698,"[{'is_accepted': True, 'is_oa': True, 'is_publ..."
22,https://openalex.org/W2965771657,"[{'is_accepted': True, 'is_oa': True, 'is_publ..."
23,https://openalex.org/W4200079077,"[{'is_accepted': True, 'is_oa': True, 'is_publ..."


In [6]:
df_work_authorship = df_work_raw.explode('locations').reset_index(drop=True)

In [7]:
def land_work2location_openalex(df_work_raw):

    df_work_location = df_work_raw.explode('locations').reset_index(drop=True)

    df_work_location = pd.concat([df_work_location['id'], json_normalize(df_work_location['locations'])], axis=1)

    df_work_location.columns = df_work_location.columns.str.replace('.', '_')

    df_work_location = df_work_location[[
        'id', 
        'source_id', 'source_display_name', 'source_is_core', 'source_type',
        'source_host_organization', 'source_host_organization_name',
        'is_accepted', 'is_oa', 'is_published', 'landing_page_url',
        'license', 'license_id', 'pdf_url', 'version',
        # 'source_host_organization_lineage', 'source_host_organization_lineage_names', 'source_issn',
        'source_is_in_doaj', 'source_is_oa', 'source_issn_l'
    ]]

    df_work_location['load_datetime'] = date.today()
    df_work_location.rename(columns={'id':'work_id'}, inplace=True)

    return df_work_location

In [8]:
stage_work_location = land_work2location_openalex(df_work_raw)
stage_work_location

[10/15/24 09:24:40] WARNING  /tmp/ipykernel_55432/1923258824.py:7: FutureWarning: The default value ]8;id=473880;file:///usr/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=994548;file:///usr/lib/python3.10/warnings.py#109\109]8;;\
                             of regex will change from True to False in a future version. In                       
                             addition, single character regular expressions will *not* be treated                  
                             as literal strings when regex=True.                                                   
                               df_work_location.columns = df_work_location.columns.str.replace('.',                
                             '_')                                                                                  
                                                                                                                   

,work_id,source_id,source_display_name,source_is_core,source_type,source_host_organization,source_host_organization_name,is_accepted,is_oa,is_published,landing_page_url,license,license_id,pdf_url,version,source_is_in_doaj,source_is_oa,source_issn_l,load_datetime
0,https://openalex.org/W4211046286,https://openalex.org/S116045149,Journal of Instrumentation,True,journal,https://openalex.org/P4310311669,Institute of Physics,True,True,True,https://doi.org/10.1088/1748-0221/3/08/s08003,None,None,https://iopscience.iop.org/article/10.1088/174...,publishedVersion,False,False,1748-0221,2024-10-15
1,https://openalex.org/W4211046286,https://openalex.org/S4306400393,Deep Blue (University of Michigan),False,repository,https://openalex.org/I27837315,University of Michigan–Ann Arbor,False,True,False,https://hdl.handle.net/2027.42/64167,None,None,http://deepblue.lib.umich.edu/bitstream/2027.4...,submittedVersion,False,True,None,2024-10-15
2,https://openalex.org/W4211046286,https://openalex.org/S4306401634,GoeScholar The Publication Server of the Geor...,False,repository,https://openalex.org/I4210122495,Asklepios Klinik St. Georg,True,True,True,http://resolver.sub.uni-goettingen.de/purl?gs-...,cc-by,https://openalex.org/licenses/cc-by,https://publications.goettingen-research-onlin...,publishedVersion,False,True,None,2024-10-15
3,https://openalex.org/W4211046286,https://openalex.org/S4306402455,CONICET Digital (CONICET),False,repository,https://openalex.org/I4210148385,Biomedicine Research Institute of Buenos Aires...,True,True,True,http://hdl.handle.net/11336/64823,cc-by-nc-sa,https://openalex.org/licenses/cc-by-nc-sa,https://ri.conicet.gov.ar/bitstream/11336/6482...,publishedVersion,False,True,None,2024-10-15
4,https://openalex.org/W4211046286,https://openalex.org/S4306400115,eScholarship (California Digital Library),False,repository,https://openalex.org/I2801248553,California Digital Library,False,True,False,https://escholarship.org/uc/item/9b14n7g1,cc-by,https://openalex.org/licenses/cc-by,https://escholarship.org/content/qt9b14n7g1/qt...,submittedVersion,False,True,None,2024-10-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1499,https://openalex.org/W4200079077,https://openalex.org/S4306400453,mediaTUM – the media and publications reposito...,False,repository,https://openalex.org/I62916508,Technical University of Munich,False,True,False,https://mediatum.ub.tum.de/doc/1654727/documen...,None,None,https://mediatum.ub.tum.de/doc/1654727/documen...,submittedVersion,False,True,None,2024-10-15
1500,https://openalex.org/W4200079077,https://openalex.org/S4306400120,Common Library Network (Der Gemeinsame Bibliot...,False,repository,https://openalex.org/I4210149878,Verbundzentrale des GBV,False,True,False,https://www.db-thueringen.de/servlets/MCRFileN...,None,None,https://www.db-thueringen.de/servlets/MCRFileN...,submittedVersion,False,True,None,2024-10-15
1501,https://openalex.org/W4200079077,https://openalex.org/S4306525036,PubMed,False,repository,https://openalex.org/I1299303238,National Institutes of Health,False,False,False,https://pubmed.ncbi.nlm.nih.gov/34910301,None,None,None,None,False,False,None,2024-10-15
1502,https://openalex.org/W2285488823,https://openalex.org/S21442059,Bioresource Technology,True,journal,https://openalex.org/P4310320990,Elsevier BV,False,False,False,https://doi.org/10.1016/j.biortech.2016.02.071,None,None,None,None,False,False,0960-8524,2024-10-15
